In [1]:
from ollama import chat
from ollama import ChatResponse
import torch
import json
import os

In [2]:
def load_json(filepath):
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filepath}")
        return None

In [1]:
def save_json(data, filepath):
    try:
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Data saved successfully to {filepath}")
    except Exception as e:
        print(f"Error saving data to {filepath}: {e}")

In [4]:
system_prompt_dict = {}

In [5]:
system_prompt_dict["Rust"] = """
You are an expert Rust code analyst. Your task is to dissect a given Rust code snippet and provide a structured JSON response detailing all of its components and overall functionality. You must strictly adhere to the following JSON format:
{
  "output": {
    "programming_language": "Rust",
    "components": [
      {
        "component_type": "<TYPE_OF_RUST_COMPONENT>",
        "component_name": "<NAME_OF_COMPONENT_IF_APPLICABLE>",
        "component_code": "<THE_ACTUAL_RUST_CODE_OF_THE_COMPONENT>",
        "component_description": "<DETAILED_DESCRIPTION_OF_COMPONENT_FUNCTIONALITY>"
      },
      { /* ... more components ... */ }
    ],
    "overall_description": "<DETAILED_SUMMARY/DESCRIPTION_OF_THE_ENTIRE_RUST_CODE_FUNCTIONALITY>"
  }
}

**Instructions:**
1. Identify the Programming Language: The script is written in Rust.
2. Component Types: Use appropriate component types, for example, MODULE_DECLARATION, USE_STATEMENT, STRUCT_DEFINITION, ENUM_DEFINITION, TRAIT_DEFINITION, FUNCTION_DEFINITION, IMPLEMENTATION_BLOCK, MATCH_STATEMENT, LET_STATEMENT, LOOP, IF_STATEMENT, PATTERN_MATCHING.
3. Component Names: Provide the correct identifier for each component (function name, struct name, enum name, variable name, etc.), or NULL if not applicable.
4. Component Code: Include the complete, unmodified Rust code for each component.
5. Component Descriptions: Provide a detailed, technical explanation of what each component does, including ownership, borrowing, pattern matching, and concurrency aspects of Rust.
6. Overall Description: Provide a detailed summary of the entire Rust code, explaining its purpose, architecture, and how components interact.
7. Strict JSON Output: Your ENTIRE response must be ONLY the valid JSON object. Do not include any explanations, introductions, or additional text outside the JSON structure.

Analyze the following Rust code properly and return ONLY the JSON response with no additional text:
"""

In [6]:
def string_to_json(input_string):
    try:
        data = json.loads(input_string)
        return data, None
    except json.JSONDecodeError as e:
        return None, str(e)

In [7]:
def make_data(data, system_prompt):
    output_data = {}

    for key, value in data.items():
        model_name = "qwen2.5-coder:32b"

        response: ChatResponse = chat(model=model_name, messages=[
            {
                'role': 'system',
                'content': system_prompt,
            },
            { 
                'role': 'user',
                'content': value,
            }
        ])

        output_data[key] = {
            "input" : value,
            "output" : response['message']['content']
        }

        print("Processed:", key)
            
    return output_data

In [8]:
data = load_json("rust_train.json")

num_keys = 25
if not isinstance(data, dict):
    raise TypeError("Input must be a dictionary.")

if not isinstance(num_keys, int) or num_keys < 0:
    raise ValueError("num_keys must be a non-negative integer.")

sliced_dict = {}
count = 0
for key, value in data.items():
    if count < num_keys:
        sliced_dict[key] = value
        count += 1
    else:
        break

del data
del count

In [9]:
print("Length of sliced dictionary:", len(sliced_dict))

Length of sliced dictionary: 25


In [10]:
output = make_data(sliced_dict, system_prompt_dict["Rust"])

Processed: rust_0
Processed: rust_1
Processed: rust_2
Processed: rust_3
Processed: rust_4
Processed: rust_5
Processed: rust_6
Processed: rust_7
Processed: rust_8
Processed: rust_9
Processed: rust_10
Processed: rust_11
Processed: rust_12
Processed: rust_13
Processed: rust_14
Processed: rust_15
Processed: rust_16
Processed: rust_17
Processed: rust_18
Processed: rust_19
Processed: rust_20
Processed: rust_21
Processed: rust_22
Processed: rust_23
Processed: rust_24


In [11]:
for k, v in output.items():
    print(f"Input: \n{v['input']}\n")
    print(f"Output: \n{v['output']}\n")
    print("-" * 80)

Input: 
use assert_cmd::prelude::*;
use dir_diff;
use std::process::Command;
use tempdir;
use tempdir::TempDir;

#[test]
fn test_go_help() {
    let path = if cfg!(targe_os = "windows") {
        "../gen-license-go/bin/windows/gen-license-go.exe"
    } else if cfg!(target_os = "linux") {
        "../gen-license-go/bin/linux/gen-license-go"
    } else {
        "../gen-license-go/bin/osx/gen-license-go"
    };
    let mut cmd = Command::new(&path);
    cmd.arg("-h");
    // assert_eq!(output, help);
    let assert = cmd.assert();
    assert.success().stdout(
        r#"gen-license-go is a 996.icu license generator implemented in Go,
this generator is developed to generate various open-source licenses including MIT, Apache, etc.
More importantly, the main purpose of this tool is to incorporate those aforesaid licenses into
a brand new license: 996.icu, defined by this repository.

Usage:
  gen-license-go [flags]
  gen-license-go [command]

Available Commands:
  gen         gen is a 996.i

In [12]:
import pickle
# Save the output to a pickle file
with open("data.pickle", "wb") as file:
    pickle.dump(output, file)

In [2]:
import pickle
# Load the output from the pickle file
with open("data.pickle", "rb") as file:
    loaded_output = pickle.load(file)
print("Loaded output from pickle file:")
for i, (k, v) in enumerate(loaded_output.items()):
    print(f"Input {i}: \n{v['input']}\n")
    print(f"Output {i}: \n{v['output']}\n")
    print("-" * 80)

Loaded output from pickle file:
Input 0: 
use assert_cmd::prelude::*;
use dir_diff;
use std::process::Command;
use tempdir;
use tempdir::TempDir;

#[test]
fn test_go_help() {
    let path = if cfg!(targe_os = "windows") {
        "../gen-license-go/bin/windows/gen-license-go.exe"
    } else if cfg!(target_os = "linux") {
        "../gen-license-go/bin/linux/gen-license-go"
    } else {
        "../gen-license-go/bin/osx/gen-license-go"
    };
    let mut cmd = Command::new(&path);
    cmd.arg("-h");
    // assert_eq!(output, help);
    let assert = cmd.assert();
    assert.success().stdout(
        r#"gen-license-go is a 996.icu license generator implemented in Go,
this generator is developed to generate various open-source licenses including MIT, Apache, etc.
More importantly, the main purpose of this tool is to incorporate those aforesaid licenses into
a brand new license: 996.icu, defined by this repository.

Usage:
  gen-license-go [flags]
  gen-license-go [command]

Available Comm

In [4]:
loaded_output["rust_19"]["output"] = loaded_output["rust_19"]["output"][:loaded_output["rust_19"]["output"].index("The JSON data you've provided contains a list")-2]
loaded_output["rust_19"]["output"] = loaded_output["rust_19"]["output"] + "\n  }\n}"


In [3]:
loaded_output["rust_17"]["output"] = loaded_output["rust_17"]["output"] + "\n}"

In [5]:
# After manual verification

final_data = {}
remove_ind_num = [0]
for i, (k, v) in enumerate(loaded_output.items()):
    if i not in remove_ind_num:
        print(f"Keeping index {i} in output.")
        final_data[k] = v

Keeping index 1 in output.
Keeping index 2 in output.
Keeping index 3 in output.
Keeping index 4 in output.
Keeping index 5 in output.
Keeping index 6 in output.
Keeping index 7 in output.
Keeping index 8 in output.
Keeping index 9 in output.
Keeping index 10 in output.
Keeping index 11 in output.
Keeping index 12 in output.
Keeping index 13 in output.
Keeping index 14 in output.
Keeping index 15 in output.
Keeping index 16 in output.
Keeping index 17 in output.
Keeping index 18 in output.
Keeping index 19 in output.
Keeping index 20 in output.
Keeping index 21 in output.
Keeping index 22 in output.
Keeping index 23 in output.
Keeping index 24 in output.


In [6]:
final_data.keys()

dict_keys(['rust_1', 'rust_2', 'rust_3', 'rust_4', 'rust_5', 'rust_6', 'rust_7', 'rust_8', 'rust_9', 'rust_10', 'rust_11', 'rust_12', 'rust_13', 'rust_14', 'rust_15', 'rust_16', 'rust_17', 'rust_18', 'rust_19', 'rust_20', 'rust_21', 'rust_22', 'rust_23', 'rust_24'])

In [7]:
# Save the output to a JSON file
import json
output_json_path = "final_rust_data.json"
try:
    save_json(final_data, output_json_path)
except Exception as e:
    print(f"Error saving final data to JSON: {e}")

Data saved successfully to final_rust_data.json
